# پرسش ۵

برای این قسمت، ابتدا با استفاده از مدل $\text{SVM}$
به ازای هر یک از کلاس‌ها، یک کلیسفایر می‌سازیم و برای پیش‌بینی یک داده نیز، آن را با همه کلسیفایر‌ها ارزیابی می‌کنیم و ارزیاب‌ای که بیشترین اطمینان را دارد، به عنوان کلاس داده انتخاب می‌شود۰  
به منظور جلوگیری از بیش‌برازش، پارامتر‌های مسئله را با $\text{k-fold validation}$ 
روی دسته $\text{train}$
تعیین می‌کنیم؛ به گونه‌ای که واریانس دقت کم و میانگین دقت زیاد شود۰  
برای بررسی بیش‌برازش شدن نیز می‌توانیم $\text{train error}$ 
را با $\text{test error}$
مقایسه کنیم. در صورتی که تفاوت این دو مقدار زیاد باشد و خطای تست خیلی کمتر باشد، بیش‌برازش رخ داده‌است۰

ابتدا داده‌ها را لود می‌کنیم و در هم میریزیم۰

In [1]:
using LinearAlgebra, DelimitedFiles, Plots, MLJ, LIBSVM, Printf

data = readdlm("./data/data5.txt")
sfl = shuffle(1:size(data)[1])
lbl = Int.(data[sfl, end])
data = data[sfl, 1:end-1]

2126×21 Array{Float64,2}:
 119.0  0.002  0.0    0.004  0.0    0.0  …  121.0  122.0  123.0    2.0   0.0
 128.0  0.006  0.006  0.005  0.004  0.0     133.0  133.0  135.0   16.0   1.0
 128.0  0.009  0.0    0.007  0.002  0.0     133.0  147.0  140.0  100.0   0.0
 150.0  0.0    0.0    0.0    0.0    0.0     154.0  153.0  155.0    0.0   1.0
 122.0  0.003  0.0    0.008  0.008  0.0     127.0  120.0  124.0   28.0   0.0
 122.0  0.004  0.0    0.005  0.004  0.0  …  133.0  122.0  127.0   26.0   1.0
 138.0  0.0    0.0    0.005  0.0    0.0     141.0  140.0  142.0    1.0   0.0
 129.0  0.0    0.018  0.0    0.0    0.0     133.0  133.0  134.0    1.0   0.0
 130.0  0.002  0.002  0.009  0.009  0.0      60.0   88.0  113.0  254.0  -1.0
 136.0  0.0    0.0    0.006  0.003  0.0     140.0  137.0  140.0   12.0   1.0
 120.0  0.009  0.091  0.002  0.0    0.0  …  163.0  152.0  160.0   42.0   1.0
 148.0  0.0    0.0    0.0    0.0    0.0     150.0  149.0  151.0    0.0   1.0
 148.0  0.0    0.005  0.002  0.0    0.0     150.0 

حال داده‌ها را به دو قسمت تست و آموزش تقسیم می‌کنیم

In [2]:
function get_slice(arr, k, i)
    slr = zeros(Bool, (size(arr)[1]))
    slr[(size(arr)[1]÷k*(i-1)+1):(size(arr)[1]÷k*i)] .= 1
    return arr[slr, :], arr[(!).(slr), :]
end

test_data, train_data = get_slice(data, 5, 5)
test_lbl, train_lbl = get_slice(lbl, 5, 5)

([1; 1; … ; 1; 1], [1; 1; … ; 2; 1])

In [3]:
function my_model(tr_data, tr_lbl, vld_data)
    # train
    models = []
    for m = 1:3
        m_lbl = 1 * (tr_lbl[:] .== m)
        model = svmtrain(
            collect(tr_data'),
            m_lbl,
            kernel = Kernel.RadialBasis,
            gamma = 0.001,
            cost = 10.0,
        )
        push!(models, model)
    end
    #validate
    vld_pred = ones(Int, size(vld_data)[1])
    vld_dec = zeros(size(vld_data)[1])
    for m = 1:3
        pred, dec = svmpredict(models[m], collect(vld_data'))
        dec = abs.(dec)[1, :]
        msk = (pred .> 0) .& (dec .> vld_dec)
        vld_dec[msk] = dec[msk]
        vld_pred[msk] = pred[msk] * m
    end
    return vld_pred
end

function confmat_m(ŷ, y)
    return confmat(
        coerce(vec(ŷ), OrderedFactor),
        coerce(vec(y), OrderedFactor),
    )
end

function accuracy_m(ŷ, y)
    return accuracy(
        coerce(vec(ŷ), OrderedFactor),
        coerce(vec(y), OrderedFactor),
    )
end

accs = []
for fi = 1:5
    vld_data, tr_data = get_slice(train_data, 5, fi)
    vld_lbl, tr_lbl = get_slice(train_lbl, 5, fi)

    vld_pred = my_model(tr_data, tr_lbl, vld_data)
    acc = accuracy_m(vld_pred, vld_lbl) * 100

    push!(accs, acc)
    println("Fold-$fi Accuracy: $acc")
end
println("Avg Accuracy: ", sum(accs) / 5)

Fold-1 Accuracy: 91.47058823529412
Fold-2 Accuracy: 90.58823529411765
Fold-3 Accuracy: 90.88235294117646
Fold-4 Accuracy: 91.76470588235294
Fold-5 Accuracy: 88.8235294117647
Avg Accuracy: 90.70588235294117


در ادامه می‌توانید میزان دقت و ماتریس سردرگمی برای داده‌های تست را مشاهده کنید

In [4]:
test_pred = my_model(train_data, train_lbl, test_data)
show(stdout, "text/plain", confmat_m(test_pred, test_lbl))
println(accuracy_m(test_pred, test_lbl) * 100)

              ┌─────────────────────────────────────────┐
              │              Ground Truth               │
┌─────────────┼─────────────┬─────────────┬─────────────┤
│  Predicted  │      1      │      2      │      3      │
├─────────────┼─────────────┼─────────────┼─────────────┤
│      1      │     326     │     17      │      3      │
├─────────────┼─────────────┼─────────────┼─────────────┤
│      2      │     16      │     36      │      1      │
├─────────────┼─────────────┼─────────────┼─────────────┤
│      3      │      0      │      2      │     24      │
└─────────────┴─────────────┴─────────────┴─────────────┘
90.8235294117647
